In [6]:
import keras
from keras import layers
import tensorflow as tf
from keras import applications, models, layers

applications.MobileNetV2

train_dataset = keras.utils.image_dataset_from_directory(
    './data',                       # Path to your dataset
    labels='inferred',             # Infer labels from folder names
    label_mode='int',              # Binary or integer labels
    batch_size=32,                 # Number of images per batch
    image_size=(256, 256),         # Resize all images to 256x256
    shuffle=True,                  # Shuffle the dataset
    validation_split=0.2,          # Reserve 20% for validation
    subset='training',             # Use this for training data
    seed=123                       # Seed to ensure consistent split
)

validation_dataset = keras.utils.image_dataset_from_directory(
    'data',                       # Same directory for validation data
    labels='inferred',            # Infer labels
    label_mode='int',             # Binary or integer labels
    batch_size=32,                # Batch size
    image_size=(256, 256),        # Resize all images
    shuffle=True,                 # Shuffle the dataset
    validation_split=0.2,         # 20% of the data for validation
    subset='validation',          # Use this for validation data
    seed=123                      # Seed to ensure consistent split
)

# Normalize the pixel values to [0, 1]
normalization_layer = layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

# Optional: Prefetch to optimize performance during training
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Build the model using MobileNetV2 as the base
base_model = applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model layers

# Build the custom model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Assuming binary classification (fire risk vs no fire risk)
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

# Unfreeze some layers of the base model and fine-tune
base_model.trainable = True
# Refreeze initial layers if needed (optional)
for layer in base_model.layers[:-40]:
    layer.trainable = False

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training (fine-tuning)
fine_tune_history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5
)

# Save the model
model.save('fire_risk_model.h5')

Found 4 files belonging to 2 classes.
Using 4 files for training.


ValueError: No images found in directory ./data. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')